클러스터 별 태그를 로드한 후 가장 유사한 클러스터의 곡들과 비교 ?
- 유사도를 어떻게 판단할 것인가?
- 유사한 클러스터를 몇 개 뽑을 것인가?

감정(슬픔/분노/기쁨/중립/걱정) + 키워드

In [111]:
import sqlite3
import numpy as np
from gensim.models import FastText

In [159]:
class Music_Recommender:
    def __init__(self, 
                 DB = '../flask_api/db.db',
                 fasttext_PATH = './model/trained_fasttext.model'):
        self.DB = DB
        self.model = FastText.load(fasttext_PATH)
    
    def connect2db(self):
        try:
            conn = sqlite3.connect(self.DB)
            cur = conn.cursor()
        except:
            print("DB Connection Error!")
            return None, None
        return conn, cur 
    
    def load_cluster_tags(self):
        conn, cur = self.connect2db()
        cur.execute("SELECT COUNT(*)/10 FROM CLUSTER")
        n_clusters = cur.fetchall()[0][0]
        
        cluster_tags = []
        try:
            for i in range(n_clusters):
                cur.execute("SELECT tag FROM CLUSTER WHERE label = ?", [i])
                cluster_tags.append([tag for tag, in cur.fetchall()])
        except:
            pass
        finally:
            conn.commit()
            conn.close()
        return cluster_tags
    
    def find_similar_clusters(self, keywords):
        cluster_tags = self.load_cluster_tags()
        similarities = []
        
        for cluster_tag in cluster_tags:
            sim = 0
            for tag in cluster_tag:
                for keyword in keywords:
                    sim += self.model.wv.similarity(keyword, tag)
            similarities.append(sim)
        return np.argsort(similarities)[::-1][:len(similarities)//20]

    def run(self, keywords):
        # return musics in @sim_clusters and select randomly with weight on 'like_cnt'
        sim_clusters = self.find_similar_clusters(keywords)
        conn, cur = self.connect2db()
        
        
        pass
        
        

In [160]:
recom = Music_Recommender()
recom.find_similar_clusters(['기쁨', '드라이브'])

array([ 2, 28])